In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pygcransac
from time import time

correspondences = np.loadtxt('img/pose6dscene_points.txt')
gt_pose = np.loadtxt('img/pose6dscene_gt.txt')
intrinsic_params = np.loadtxt('img/pose6dscene.K')

print("Number of correspondences loaded = ", str(len(correspondences)))

Number of correspondences loaded =  95


In [2]:
def verify_pygcransac(corrs, K):    
    n = len(correspondences)
    imagePoints = np.float32([corrs[i][0:2] for i in np.arange(n)]).reshape(-1,2)
    worldPoints = np.float32([corrs[i][2:5] for i in np.arange(n)]).reshape(-1,3)
    
    threshold = 5.5
    normalized_threshold = threshold / (K[0, 0] + K[1, 1]) / 2.0;    
    pose, mask = pygcransac.find6DPose(imagePoints, worldPoints, normalized_threshold, 0.999)    
    return pose, mask

def normalize_image_points(corrs, K): 
    n = len(corrs)
    normalized_correspondences = corrs
    inv_K = np.linalg.inv(K)

    for i in range(n):
        p1 = np.append(correspondences[i][0:2], 1)
        p2 = inv_K.dot(p1)
        normalized_correspondences[i][0:2] = p2[0:2]
    return normalized_correspondences

def calculate_error(gt_pose, est_pose):
    
    R2R1 = np.dot(gt_pose[:, 0:3].T, est_pose[:, 0:3])
    cos_angle = max(-1.0, min(1.0, 0.5 * (R2R1.trace() - 1.0)))
    
    err_R = np.arccos(cos_angle) * 180.0 / np.pi
    err_t = np.linalg.norm(gt_pose[:, 3] - est_pose[:, 3])
    
    return err_R, err_t


In [3]:
normalized_correspondences = normalize_image_points(correspondences, intrinsic_params)

t = time()

mag_pose, mag_mask = verify_pygcransac(normalized_correspondences, intrinsic_params)
print (time()-t, ' sec gc-ransac')

err_R, err_t = calculate_error(gt_pose, mag_pose)

print ('Rotation error = ', err_R, '°')
print ('Translation error = ', err_t, ' mm')

0.5245976448059082  sec gc-ransac
Rotation error =  1.8783020217535148e-05 °
Translation error =  0.0001007179771463513  mm


In [3]:
corrs = np.array([
  [174.0, 178.0, -44.40372861289978, 18.693654983043672, 68.97037449645995],
  [178.0, 178.0, -42.94194043540955, 21.301651029586793, 68.77027169036865],
  [182.0, 178.0, -41.36653555297852, 23.828399448394777, 68.61285439300536],
  [186.0, 178.0, -39.76021231079102, 26.33738639831543, 68.4661106185913],
  [190.0, 178.0, -37.8753932800293, 28.63038471221924, 68.42730656433105],
  [146.0, 182.0, -48.71257399749756, -3.815829544067384, 69.53627783966064],
  [150.0, 182.0, -47.68054612731933, -0.975803936958313, 69.1851196460724],
  [154.0, 182.0, -47.087395034790035, 2.2029190406799315, 68.66197963285447],
  [158.0, 182.0, -45.841171585083, 4.910026584625244, 68.38803096342087],
  [162.0, 182.0, -43.799498878479, 7.026360069274903, 68.42106624174119],
  [166.0, 182.0, -42.43735059356689, 9.660711799621582, 68.1891021900177],
  [138.0, 194.0, -35.662597648620604, -20.771013258695604, 67.11075592947006],
  [142.0, 194.0, -33.36214446258545, -18.95072355747223, 67.26926423025131],
  [146.0, 194.0, -30.96948622894287, -17.20125177383423, 67.4653167219162],
  [150.0, 194.0, -29.136329613685607, -15.043678726196289, 67.43852234113217],
  [154.0, 194.0, -27.904422722816467, -12.437220062255859, 67.17009733843804]
], np.float64)

In [4]:

# 2D image coordinates (in px).
corr_2d = corrs[:, :2]

# 3D model coordinates (in mm).
corr_3d = corrs[:, 2:]

# Intrinsics.
K = np.array([
  [1066.778, 0.0, 312.9869],
  [0.0, 1067.487, 241.3109],
  [0.0, 0.0, 1.0]
], np.float64)

# Inlier threshold (in px).
inlier_thresh = 4.0

In [5]:
# OpenCV PnP-RANSAC.
# ------------------------------------------------------------------------------
est_cv_success, r_est_cv, t_est_cv, inliers = cv2.solvePnPRansac(
  objectPoints=corr_3d,
  imagePoints=corr_2d,
  cameraMatrix=K,
  distCoeffs=None,
  iterationsCount=400,
  reprojectionError=inlier_thresh,
  confidence=0.99,
  flags=cv2.SOLVEPNP_EPNP)

# Rodrigues rotation vector to a 3x3 rotation matrix.
R_est_cv = cv2.Rodrigues(r_est_cv)[0]

print('----------------------')
print('OpenCV PnP-RANSAC')
print('Success: {}'.format(est_cv_success))
print('R: {}'.format(R_est_cv))
print('t: {}'.format(t_est_cv))
print(inliers.size)

----------------------
OpenCV PnP-RANSAC
Success: True
R: [[ 0.7091456   0.70483857  0.01775135]
 [ 0.25790341 -0.23588271 -0.93693392]
 [-0.65619993  0.6690007  -0.34905546]]
t: [[-86.4400753 ]
 [ 33.35438989]
 [777.05154377]]
16


In [8]:
# GC-RANSAC.
# ------------------------------------------------------------------------------
# Normalized inlier threshold on the reprojection error (used in PnP-RANSAC).
f = (K[0, 0] + K[1, 1]) / 2.0
inlier_thresh_normalized = inlier_thresh / f

# Normalize the 2D image coordinates.
K_inv = np.linalg.inv(K)
corr_2d_h = np.hstack((corr_2d, np.ones((corr_2d.shape[0], 1))))
corr_2d_normalized = K_inv[:2, :].dot(corr_2d_h.T).T

#corr_2d_normalized = np.float32([corr_2d_normalized[i][0:2] for i in np.arange(corr_2d_normalized.shape[0])]).reshape(-1,2)
corr_2d_normalized = np.array([r for r in corr_2d_normalized])

pose, mask = pygcransac.find6DPoseEPOS(
  x1y1=corr_2d_normalized.astype(np.float32),
  x2y2z2=corr_3d.astype(np.float32),
  K=K.astype(np.float32),
  threshold=inlier_thresh_normalized,
  conf=0.999999,
  spatial_coherence_weight=0.0,
  max_iters=1000000)
est_gc_success = pose is not None
R_est_gc = np.float32(pose[:3, :3])
t_est_gc = np.float32(pose[:, 3].reshape((3, 1)))

print('----------------------')
print('GC-RANSAC')
print('Success: {}'.format(est_gc_success))
print('R: {}'.format(R_est_gc))
print('t: {}'.format(t_est_gc))
print(inlier_thresh_normalized)


----------------------
GC-RANSAC
Success: True
R: [[ 0.70914537  0.7048388   0.01775099]
 [ 0.25790322 -0.23588283 -0.93693393]
 [-0.65620023  0.66900045 -0.3490554 ]]
t: [[-86.44009 ]
 [ 33.354374]
 [777.0517  ]]
0.003748363019587539
